In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
    
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    %cd /content
    !rm -r /content/NCTU_Adv_DNN_HW2_2
    !rm -r /content/exec_env
    !rm -r /content/tool
    !rm /content/test.zip
    !rm -r /content/test
    !git clone https://github.com/chilung/NCTU_Adv_DNN_HW2_2.git
    !mv /content/NCTU_Adv_DNN_HW2_2/* .
    !rm -r /content/NCTU_Adv_DNN_HW2_2
    gdd.download_file_from_google_drive(file_id='1nswVLQSGupsRympzb3tUv3L94d7ADPmi',
                dest_path='./test.zip',
                unzip=True)
    gdd.download_file_from_google_drive(file_id='1VGItDjkaoJ9Cq1RiCXnuXV7mWF930-Y1',
                dest_path='./exec_env/yolo-obj-mAP_93.3.weights',
                unzip=False)

Running on CoLab
/content
rm: cannot remove '/content/chilung-NCTU_Adv_DNN_HW2_2': No such file or directory
rm: cannot remove '/content/exec_env': No such file or directory
rm: cannot remove '/content/tool': No such file or directory
rm: cannot remove '/content/test.zip': No such file or directory
rm: cannot remove '/content/test': No such file or directory
Cloning into 'chilung-NCTU_Adv_DNN_HW2_2'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 30 (delta 3), reused 26 (delta 2), pack-reused 0
Unpacking objects: 100% (30/30), done.
Unzipping...Done.


In [ ]:
!python demo.py -cfgfile='./exec_env/yolo-obj.cfg' -weightfile='./exec_env/yolo-obj-mAP_93.3.weights' -imgfiles='./exec_env/valid.txt' -namesfile='./exec_env/obj.names'

Namespace(cfgfile='./exec_env/yolo-obj.cfg', imgfiles='./exec_env/valid.txt', namesfile='./exec_env/obj.names', weightfile='./exec_env/yolo-obj-mAP_93.3.weights')
convalution havn't activate linear
convalution havn't activate linear
convalution havn't activate linear
Loading weights from ./exec_env/yolo-obj-mAP_93.3.weights... Done!
Sat Nov 21 02:11:20 2020
predict img proceed: 0 of 13068
predict img proceed: 100 of 13068
predict img proceed: 200 of 13068
predict img proceed: 300 of 13068
predict img proceed: 400 of 13068
predict img proceed: 500 of 13068
predict img proceed: 600 of 13068
predict img proceed: 700 of 13068
predict img proceed: 800 of 13068
predict img proceed: 900 of 13068
predict img proceed: 1000 of 13068
predict img proceed: 1100 of 13068
predict img proceed: 1200 of 13068
predict img proceed: 1300 of 13068
predict img proceed: 1400 of 13068
predict img proceed: 1500 of 13068
predict img proceed: 1600 of 13068
predict img proceed: 1700 of 13068
predict img proceed: 1

In [ ]:
!cp submission.json '/content/drive/MyDrive/Colab Notebooks/NCTU_Adv_DNN_HW2/'